**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-p4tqmbtk
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-p4tqmbtk
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-p4tqmbtk did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-p4tqmbtk did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,0596c3b57c375341bbbf5c60aa37895e66d7cfe8,0083fcde0fb490b4ec424c63d1d750378f6ad11154d1d4...,2017-07-31 17:00:00+00:00,2017-07-31 17:00:00+00:00,240,0.0,NaN,NaN,35,35,...,2.5,8.00,Cash,Taxi Affiliation Services,41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863)
1,11fb1a0ce633b924b51fbb1037a702454e6a4e67,e88fa53c774c82956a2aad60dcbdd260f05b157cf197da...,2017-07-09 23:15:00+00:00,2017-07-09 23:15:00+00:00,360,0.0,NaN,NaN,21,10,...,0.0,18.75,Cash,Blue Ribbon Taxi Association Inc.,41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),41.985015,-87.804532,POINT (-87.8045320063 41.9850151008)
2,524b211fe3a58f0feae184ff23b42d004630624b,a149c6df861b2eff6f27491e024e37d2c3705aa5399c31...,2017-07-01 23:15:00+00:00,2017-07-01 23:30:00+00:00,540,0.0,1.703106e+10,1.703103e+10,6,3,...,0.0,6.75,Cash,Choice Taxi Association,41.953400,-87.646007,POINT (-87.6460070664 41.9534000435),41.958155,-87.653022,POINT (-87.6530217894 41.9581548757)
3,368230c5225fcd5f64ddd0913f496ecccba4b837,ea9c7f865233f880e5f00abb728092901eeaf52c85a8c1...,2017-07-01 19:45:00+00:00,2017-07-01 19:45:00+00:00,540,1.7,NaN,NaN,1,1,...,0.0,7.75,Cash,Choice Taxi Association,42.009623,-87.670167,POINT (-87.67016685690001 42.0096228806),42.009623,-87.670167,POINT (-87.67016685690001 42.0096228806)
4,6cf694ad332c878c61f1773416cd1ec7c96b5541,f419fe5a1711d4b01b124b8e9f3bef383be12f31cfb043...,2017-07-31 11:00:00+00:00,2017-07-31 12:15:00+00:00,4980,0.0,NaN,NaN,2,76,...,0.0,51.75,Cash,Choice Taxi Association,42.001571,-87.695013,POINT (-87.6950125892 42.001571027),41.980264,-87.913625,POINT (-87.913624596 41.9802643146)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-19,85156.571429
1,2016-02-10,91321.142857
2,2016-01-07,75520.428571
3,2016-02-21,89790.714286
4,2016-01-18,85136.000000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,26.0,2013-10-03 04:30:00+00:00,2013-10-03 04:45:00+00:00,1
1,26.0,2013-10-03 05:15:00+00:00,2013-10-03 05:15:00+00:00,2
2,14.0,2013-10-03 00:00:00+00:00,2013-10-03 00:15:00+00:00,1
3,14.0,2013-10-03 00:15:00+00:00,2013-10-03 00:30:00+00:00,2
4,14.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,05001a5e70a7a8f38a2e4d2ee8a1e0fb87a20857413b44...,2013-10-03 13:30:00+00:00,2013-10-03 14:00:00+00:00,165.0
1,0520dba051796dd5a5a6bb274310b504d97f3a3b0b7d18...,2013-10-03 14:00:00+00:00,2013-10-03 14:15:00+00:00,255.0
2,125667cf70469e4ada5ced2973ac5fa8d2dc10745c1267...,2013-10-03 21:00:00+00:00,2013-10-03 21:00:00+00:00,210.0
3,1be1407efca4ff6ac787c8794df57e8eeac031eaf08fca...,2013-10-03 21:15:00+00:00,2013-10-03 21:15:00+00:00,165.0
4,1e5b69c83640bd5225fb3ccc33bc989b859f3e4cdb59f6...,2013-10-03 14:15:00+00:00,2013-10-03 14:15:00+00:00,270.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*